In [18]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
from anthony.aes import sub_bytes, add_round_key
from scipy.stats import pearsonr
from multiprocessing import Pool
import pandas as pd

In [26]:
# Data directory
workdir = "data/dataset1"
# Number of plaintexts/traces
num_traces = 150
# Number of samples per trace
num_samples = 50000
# Number of key bytes
num_key_bytes = 16

# Initialize a 3D NumPy array to hold all traces
# Dimensions: 16 key bytes x 150 traces x 50000 samples
# all_traces = np.zeros((num_traces, num_samples*16), dtype=np.float64)
all_traces = np.zeros((16, 150, 50000), dtype=np.float64)

# Read the traces
# for i in range(num_key_bytes):
#     filename = f'{workdir}/trace{i}.txt'
#     with open(filename, 'r') as file:
#         for j, line in enumerate(file):
#             # Convert line to float values and store in array
#             all_traces[j, 50000*i:50000*(i+1)] = np.array(line.strip().split(), dtype=np.float64)

for i in range(16):
    with open(f'data/dataset1/trace{i}.txt') as f:
        for t, line in enumerate(f):
            all_traces[i][t] = np.array(line.strip().split(), dtype=np.float64)

In [30]:
plaintexts = np.zeros((num_traces, num_key_bytes), dtype=int)

# Read the plaintexts
with open(f'{workdir}/cleartext.txt', 'r') as file:
    for i, line in enumerate(file):
        # Convert line to integer values and store in array
        plaintexts[i] = np.array(line.strip().split(), dtype=int)

In [31]:
def hamming_weight(n):
    count = 0
    while n:
        count += n & 1
        n >>= 1
    return count

hamming_weights = np.array([hamming_weight(i) for i in range(256)])

In [32]:
all_hamming_weights_per_box2 = np.zeros((150, 256, 16), dtype=np.uint8)
coeff2 = np.zeros((16, 256))

for key_guess in range(256):
    # print(f'Analyzing key byte guess: {hex(key_guess)}')
    key_guess_matrix = np.full((4, 4), key_guess, dtype=np.uint8)
    for tnum, in_data in enumerate(plaintexts):
        # fancy way to parse hex data and put it into a 4x4 matrix
        in_matrix = np.asarray(in_data, dtype=np.uint8).reshape(4, 4)

        added_key = add_round_key(in_matrix, key_guess_matrix)
        sub_out = sub_bytes(added_key, False).flatten()

        for byte_index in range(sub_out.shape[0]):
            all_hamming_weights_per_box2[tnum][key_guess][byte_index] = hamming_weights[sub_out[byte_index]]

    # for box in range(16):
    #     guess_box_weights = all_hamming_weights_per_box2[:, key_guess, box]
    #     WxH = (all_traces.T * guess_box_weights).T
    #     EWxH = np.sum(WxH, axis=0)

    #     Ew = np.sum(all_traces, axis=0)
    #     Eh = np.sum(guess_box_weights)

    #     Ew2 = np.sum(np.square(all_traces), axis=0)
    #     Ew_squared = np.square(Ew)

    #     Eh2 = np.sum(np.square(guess_box_weights), axis=0)
    #     Eh_squared = np.square(Eh)

    #     top = (150 * EWxH) - (Eh * Ew)
    #     bottom = np.sqrt((150 * Ew2) - Ew_squared) * np.sqrt((150 * Eh2) - Eh_squared)
    #     coeff2[box][key_guess] = np.max(top / bottom)

In [15]:
all_hamming_weights_per_box2[:, 0, 0]

(150,)

In [33]:
def multi(box: int):
	for guess in range(256):
		guess_box_weights = all_hamming_weights_per_box2[:, guess, box]
		WxH = (all_traces[box].T * guess_box_weights).T
		EWxH = np.sum(WxH, axis=0)

		Ew = np.sum(all_traces[box], axis=0) # constant

		Eh = np.sum(guess_box_weights)

		Ew2 = np.sum(np.square(all_traces[box]), axis=0) # constant
		Ew_squared = np.square(Ew) # constant

		Eh2 = np.sum(np.square(guess_box_weights), axis=0)
		Eh_squared = np.square(Eh)

		top = (150 * EWxH) - (Eh * Ew)
		bottom = np.sqrt((150 * Ew2) - Ew_squared) * np.sqrt((150 * Eh2) - Eh_squared)
		print(f'{np.max(top/bottom)=}')
		coeff2[box][guess] = np.max(top / bottom)

In [34]:
multi(0)

np.max(top/bottom)=0.32693942294060585
np.max(top/bottom)=0.34359707452547356
np.max(top/bottom)=0.36081007555659644
np.max(top/bottom)=0.3367837243475755
np.max(top/bottom)=0.3437160176099413
np.max(top/bottom)=0.3503428169307833
np.max(top/bottom)=0.36671310514123795
np.max(top/bottom)=0.3633358544098217
np.max(top/bottom)=0.31510847352401206
np.max(top/bottom)=0.32058866909770284
np.max(top/bottom)=0.30323092739568147
np.max(top/bottom)=0.3464730112611184
np.max(top/bottom)=0.31838043198836213
np.max(top/bottom)=0.31006570799150673
np.max(top/bottom)=0.3473192306883835
np.max(top/bottom)=0.33797276215012756
np.max(top/bottom)=0.3202237911025384
np.max(top/bottom)=0.3399237492773698
np.max(top/bottom)=0.31560770210827394
np.max(top/bottom)=0.33499441334814634
np.max(top/bottom)=0.339893419293925
np.max(top/bottom)=0.35503312525405384
np.max(top/bottom)=0.32723355773130736
np.max(top/bottom)=0.35717569683374395
np.max(top/bottom)=0.3395739354584759
np.max(top/bottom)=0.341951007893327

In [13]:
np.argsort(coeff2[0])

array([207,  20,  90, 105, 195, 124,  24, 242,  59,  70,  85,  30,  42,
       157, 180,  78,  41, 247,  25, 181, 103,  43, 254, 227,  55, 228,
       221, 205, 158, 204, 131,  84, 206,  89, 114, 109,   9,  93, 126,
       113, 224, 145, 130, 170, 239,  64, 235, 169,  62,  76, 193,  46,
       154, 191,  11, 102, 160, 234, 244, 230,  50, 183, 231, 214,  32,
       132, 164, 116,  72, 107,  21,  14, 211, 232, 112, 127, 238, 100,
        54,  80,  18, 163, 140, 165,  87, 128,  58, 168,  44, 141,  29,
       119,   1,  13, 121,  95,  92, 144, 136,  47,  66,  69, 137,   2,
       189, 243, 223,   4, 212, 174,  15,   7, 153, 139,   8, 148, 251,
       150, 101,  99, 166, 213, 186, 201, 151, 135,  35, 104, 192, 237,
        28,  68, 146,  33, 118, 202, 184, 133,  74, 210, 125, 203,  81,
        40, 175, 252,  83,  19, 143, 138,  75, 246, 200, 253, 194, 220,
       173,  63, 106,  98, 245,  16, 115,  22, 161, 123, 129, 171, 225,
       229,  79,  23, 250, 249, 177,  77, 222,  60, 176,  96,  5

In [ ]:
# for guess in range(256):
#     print(f'{guess=}')
#     for box in range(1):
#         guess_box_weights = all_hamming_weights_per_box2[:, guess, box]
#         WxH = (all_traces.T * guess_box_weights).T
#         EWxH = np.sum(WxH, axis=0)

#         Ew = np.sum(all_traces, axis=0) # constant
#         Eh = np.sum(guess_box_weights)

#         Ew2 = np.sum(np.square(all_traces), axis=0) # constant
#         Ew_squared = np.square(Ew) # constant

#         Eh2 = np.sum(np.square(guess_box_weights), axis=0)
#         Eh_squared = np.square(Eh)

#         top = (150 * EWxH) - (Eh * Ew)
#         bottom = np.sqrt((150 * Ew2) - Ew_squared) * np.sqrt((150 * Eh2) - Eh_squared)
#         coeff2[box][guess] = np.max(top / bottom)

In [35]:
np.set_printoptions(formatter={'hex':int})

for box in range(16):
    print(np.argsort(coeff2[box, :])[-1])

65
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255


In [16]:
np.set_printoptions(formatter={'hex':int})

for box in range(16):
    print(np.argsort(coeff2[box, :])[-1])

0x41
0x75
0x73
0x74
0x72
0x61
0x6c
0x6f
0x70
0x69
0x74
0x68
0x65
0x63
0x75
0x73
